This is my attempt at writing simulation code for Lennard-Jones. There is some stuff I incorporated from open source code by Gianmarc Grazioli - all of it is highlighted with comments of exclamation marks above and below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# define the lennard-jones force in terms of r, epsilon, sigma
#     for sufficiently large r (with periodic boundary conditions, larger than half the cell size) 
# can assume the force is zero, makes computation much easier
# force is obtained by differentiating potential wrt r

def ljp(r, epsilon, sigma):
    return 48 * epsilon * np.power(sigma, 12) / np.power(r, 13) \
    - 24 * epsilon * np.power(sigma, 6) / np.power(r, 7)

# things to make a plot of the force vs distance

r = np.linspace(3.5, 8, 100)
plt.plot(r, ljp(r, 0.0103, 3.3))
plt.xlabel('distance')
plt.ylabel('force')
plt.show()

In [ ]:
# MATH TOOLS (Gianmarc Grazioli)
# Originally I wasn't using these, but they're much more efficient than what I had so I've incorporated them

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 14:43:33 2021

@author: Gianmarc Grazioli
"""

import numpy as np

def GetRandomUnitVectors(dim, count):
    randos = np.random.normal(0, 1, (dim+1)*count).reshape((count, dim+1))
    norms = [np.sqrt(sum(randos[r]**2)) for r in range(len(randos))]
    raw = [randos[i][0:dim]/norms[i] for i in range(len(norms))]
    outNorms = [np.sqrt(sum(raw[r]**2)) for r in range(len(raw))]
    out = [raw[i]/outNorms[i] for i in range(len(outNorms))]
    return(out)
    
def PeriodicVectorCorrection(vec, sideLen): 
    out = [v for v in vec]
    for i in range(len(vec)):
        if vec[i] >= .5*sideLen:
            out[i] -= sideLen
        elif vec[i] < -.5*sideLen:
            out[i] += sideLen
    return (np.array(out))
 
def getLargerPerfectSquare(num):
    i = 0
    diffSq = 9999999
    oldDiffSq = 9999999999
    while diffSq < oldDiffSq:
        oldDiffSq = diffSq
        i += 1
        diffSq = (num - i*i) * (num - i*i)
    if (i-1)*(i-1) >= num:
        out = (i-1)
    else:
        out = i
    return (out)

def getLargerPerfectCube(num):
    i = 0
    diffSq = 9999999
    oldDiffSq = 9999999999
    while diffSq < oldDiffSq:
        oldDiffSq = diffSq
        i += 1
        diffSq = (num - i*i*i) * (num - i*i*i) 
    if (i-1)*(i-1)*(i-1) >= num:
        out = (i-1)
    else:
        out = i
    return (out)

def GetPointsOnSquareGrid(num, sideLen, buffer):
    sideCount = getLargerPerfectSquare(num)
    sideGrid = np.linspace(0+buffer, sideLen-buffer, sideCount)
    fullGrid = []
    for i in sideGrid:
        for j in sideGrid:
            fullGrid.append([i,j])
    return (np.array(fullGrid[0:num]))

def GetPointsOnCubicGrid(num, sideLen, buffer):
    sideCount = getLargerPerfectCube(num)
    sideGrid = np.linspace(0+buffer, sideLen-buffer, sideCount)
    fullGrid = []
    for i in sideGrid:
        for j in sideGrid:
            for k in sideGrid:
                fullGrid.append([i,j,k])
    return (np.array(fullGrid[0:num]))

In [ ]:
# CLASSES

# Define molecule class for particles
class Molecule():
    def __init__(self, r, v, a, m):
        self.r = r   # position vector 
        self.v = v   # velocity vector
        self.a = a   # acceleration vector
        self.m = m   # mass
        
# Define system class for whole system     
class System():
    def __init__(self, sideLen=10, dim=2, buffer=0.75, nMols=36, temperature=1, sigma=1, epsilon=1, mass=1, cutoff=3, gravity=9.8):
        # define what the class will refer to its properties as
        self.bounds = [(0,sideLen) for i in range(dim)]   # define the walls of the container
        self.temperature = temperature                    # temperature of the system
        self.sideLen = sideLen                           # side length of container
        self.dim = dim                                    # dimension of the simulation (2 or 3)
        self.sigma = sigma                                # sigma parameter of LJ
        self.epsilon = epsilon                            # epsilon parameter of LJ
        self.nMols = nMols                                # number of molecules
        self.cutoff = cutoff                              # distance after which to set potential to 0
        self.gravity = gravity                            # value of gravitational acceleration. set to 0 for none.
        
        # define positions, depending on dimension
        match dim:
            case 2: 
                # 2 dimensional, we will initialise positions to the vertices on a square grid
                # below code defines an array of points. 
                # number of points = nMols, in a square of side length sideLen, with buffer distance from edge
                # note: I couldn't find documentation for this library so I don't know how it handles
                # when nMols is not a square number. I will restrict myself to square numbers then
                positions = GetPointsOnSquareGrid(nMols, sideLen, buffer)
            
            case 3: 
                # basically the same as for 2 dimensions, but this time we want nMols to be a cube
                positions = GetPointsOnCubicGrid(nMols, sideLen, buffer)
                
            case _:
                print("No dimension added!! :(")
                
        # set initial velocities and accelerations to 0 in each direction for 0 initial values overall
        # velocity will be updated to random values later
        v = np.zeros(dim)
        a = np.zeros(dim)
        
        # create a list of each Molecule() instance
        self.molecules = []
        
        # give the molecules their properties
        for p in positions:
            self.molecules.append(Molecule(p, v, a, mass))
            
        # calculate total mass inside system as mass times number of molecules.
        totalMass = mass * len(self.molecules)
        
        # calculate some properties
        self.volume = sideLen**dim                   # volume
        self.density = totalMass/(self.volume)      # density of whole system
        self.posTraj = [np.vstack(positions)]        # record positions at each step
        self.velTraj = [np.vstack([molecule.v for molecule in self.molecules])]          # record velocities at each step
        # pressure and instantaneous pressures will be calculated later, but it helps to initialise type
        self.instPressures = []
        self.pressure = 0
        
        # initialise velocities. see function below
        self.InitialiseVelocities()
        
    def InitialiseVelocities(self): 
        # get random directions, with dimension dim, and get nMols of them
        randUnitVecs = GetRandomUnitVectors(self.dim, self.nMols)
        
        # initialise a velocity sum, will be helpful for centre of mass
        velSum = np.zeros(self.dim)
        
        # calculate velocity magnitude according to temperature and mass.
        # use np array datatype because it is convenient, but really all the entries are the same
        # use kinetic theory of gases. 1/2 mv^2 = (dim)/2 kB T. for convenience, set kB = 1.
        velMags = [np.sqrt(self.dim * self.temperature / molecule.m) for molecule in self.molecules]
        
        # use enumerate to do a for loop and make the iteration number available in one step
        for i, molecule in enumerate(self.molecules):
            molecule.v = velMags[i] * randUnitVecs[i]
            velSum += molecule.v
        
        # calculate where the centre of velocity vectors is (i.e. average the vectors over all molecules)
        vCentrepoint = velSum/self.nMols
        # subtract centrepoint from each velocity to make the centre of mass stationary in this reference frame
        for molecule in self.molecules:
            molecule.v -= vCentrepoint
            
        
    # fix positions for periodic boundaries. basically a wrapping function
    # note: look into optimising. should I get rid of the check and just do mod on everything? basically, does it take more computing power to mod everything or check everything then mod
    def FixPeriodicPositions(self):
        for molecule in self.molecules:
            # get the position from the molecule instance
            position = molecule.r
            
            # check if molecule is out of bounds in each dimension.
            for d in range(self.dim):
                # check if out of bounds
                if position[d] > self.sideLen or position[d] < 0:
                    # update position to be less than side length using modulo operator
                    position[d] = position[d] % self.sideLen
                    
    # figure out and add forces between two molecules due to LJ potential
    def AddLJForce(self, molecule1, molecule2):
        mol1, mol2 = molecule1, molecule2
        distance = mol2.r - mol1.r
        
        # correct distance to account for periodic boundary conditions. from mathTools
        distance = PeriodicVectorCorrection(distance, self.sideLen)
        
        # get magnitude of distance
        r = np.sqrt(sum(distance**2))
        
        # store sigma^6 and r^7 to make computation easier using the fact that we also need sigma^12 and r^13
        # used r^6 because dividing by r is more expensive than multiplying by r later
        sigma6 = self.sigma**6
        sigma12 = sigma6**2
        r6 = r**6
        r7 = r6 * r
        r13 = r6**2 * r
        
        # calculate force
        LJforce = 48 * self.epsilon * (sigma12/r13) - 24 * self.epsilon * (sigma6/r7)
        
        # add acceleration due to force
        molecule1.a -= LJforce * (distance/r) / molecule1.m
        molecule2.a += LJforce * (distance/r) / molecule2.m
                    
    # figure out and add force of gravity to each molecule
    def AddGravity(self, molecule):
#         molecule = self.molecules[molecule]
        g = self.gravity
        
        Gforce = molecule.m * g
        
        molecule.a = np.add(molecule.a, Gforce)
        
    # update accelerations
    def UpdateAcceleration(self):
        
        radiusCutSquare = self.cutoff**2     # square of the cutoff radius
        virSum = 0                           # initialise virial sum to find virial pressure
        vdotv = 0                            # initialise dot product of velocities to find temperature
        
        masses = [self.molecules[i].m for i in range(0, len(self.molecules))]
        
        # reset acceleration to 0 for this new step
        for molecule in self.molecules:
            molecule.a = np.zeros(self.dim)
            vdotv += np.dot(molecule.v, molecule.v) * molecule.m
            
        # loop over every molecule to update its acceleration
        for molecule in self.molecules:
            molecule.a = np.zeros(self.dim)         # reset acceleration to 0
            vdotv += np.dot(molecule.v, molecule.v) * molecule.m
            
#             for j in range(self.nMols):
            for molecule2 in self.molecules:
                molecule1 = molecule
                distance = molecule2.r - molecule1.r
                distance = PeriodicVectorCorrection(distance, self.sideLen)
                distanceSquare = sum(distance**2)
                
                # check distance is less than cutoff. if yes, find and add potentials from LJ
                if ( (distanceSquare < radiusCutSquare) and (self.molecules.index(molecule1) < self.molecules.index(molecule2)) ):
                    self.AddLJForce(molecule1,molecule2)
                    virSum += np.dot(distance, molecule.a)
                    
            self.AddGravity(molecule)
                    
        # update some system properties
        self.temperature = vdotv/(self.nMols * self.dim)
        V, T, rho = self.volume, self.temperature, self.density
        P = rho*T + 1/self.dim * virSum/V
        self.instPressures.append(P)
        

    def onePairLJenergy(self, mol1, mol2): 
        # returns Lennard-Jones potential energy between one pair of molecules 
        m1, m2 = self.moList[mol1], self.moList[mol2] 
        diff = m2.r - m1.r 
        if self.periodic:
            diff = PeriodicVectorCorrection(diff, self.sideLen)
        r = np.sqrt(sum(diff**2)) 
        sigOverR6 = np.power(self.sigma/r, 6)
        sigOverR12 = sigOverR6*sigOverR6        
        return (4 * self.epsilon * (sigOverR12 - sigOverR6))
    
    def getTotalPE(self):
        # get total potential energy 
        totalPE = 0
        radiusCutSquare = self.cutoff**2
        
#         distance = molecule2.r - molecule1.r
#         r = np.sqrt(sum(distance**2))
#         sigmaOnRMinus6 = np.power(self.sigma/r, 6)
#         sigmaOnRMinus12 = sigmaOnRMinus6**2
        
        for i in range(0, self.nMols):
            for j in range(0, self.nMols):
                molecule1, molecule2 = self.molecules[1], self.molecules[2]
                distance = molecule2.r - molecule1.r
                distance = PeriodicVectorCorrection(distance, self.sideLen)
                r = np.sqrt(sum(distance**2))
                distanceSquare = sum(distance**2)
                
                sigmaOnRMinus6 = np.power(self.sigma/r, 6)
                sigmaOnRMinus12 = sigmaOnRMinus6**2
                
                # check distance is less than cutoff. if yes, find and add potentials from LJ
                if ( (distanceSquare < radiusCutSquare) and (i < j) ):
                    totalPE += 4 * self.epsilon * (sigmaOnRMinus12 - sigmaOnRMinus6)
        print(totalPE)
        return (totalPE)
                
    def getTotalKE(self):
        vdotvTotal = 0
        for molecule in self.molecules:
            vdotvTotal += 0.5 * molecule.m * sum(molecule.v**2)
        return (vdotvTotal)

In [ ]:
# DYNAMICS
from numpy import vstack, mean

def SingleStep(system, dt=0.005):
    
    for i in range(0, len((system.molecules))):
        oldAccelerations = system.molecules[i].a.copy()
        print(system.molecules[i].r)
    
    system.FixPeriodicPositions()
    system.UpdateAcceleration()
    
    # update velocities with old acceleration
    for i, molecule in enumerate(system.molecules):
            molecule.v = molecule.v +  0.5*(molecule.a + oldAccelerations[i])*dt

            

def RunDynamics(system, nSteps = 1000, dt=.005, saveFreq=5):
    KE = []
    PE = []
    for t in range(0, nSteps):
        if (t % saveFreq == 0):
            positions = [system.molecules[i].r for i in range(0, len(system.molecules))]
            velocities = [system.molecules[i].v for i in range(0, len(system.molecules))]
            system.posTraj.append(vstack(positions))
            system.velTraj.append(vstack(velocities))
            KE.append(system.getTotalKE())
            PE.append(system.getTotalPE())
        
        SingleStep(system, dt=dt)
        system.pressure = mean(system.instPressures) 
        if (t % 100 == 0): print("Simulation step",t,"complete")
    
    with open("KE.csv", "w") as KEfile:
        KEfile.write("t,KE\n") 
        for idx,k in enumerate(KE):
            KEfile.write(str(idx * saveFreq) + ", " + str(k)+"\n")
    with open("PE.csv", "w") as PEfile:
        PEfile.write("t,PE\n")
        for idx,p in enumerate(PE):
            PEfile.write(str(idx * saveFreq) + ", " + str(p)+"\n")

In [ ]:
import time
import visualizationTools as vis

testSim = System() 

startTime = time.time() 

# run the dynamics for the simulation. 1500 steps, saving data every 5 steps, to give 300 data points
RunDynamics(testSim, nSteps=1000, saveFreq=5)

# get and print the total execution time
executionTime = (time.time() - startTime)
print('Simulation time: ' + str(int(executionTime / 60)) + " min and ", str(int(executionTime) % 60) + " seconds.")

# # make gif, which will appear in the directory this was run in
# print("Creating gif")
# vis.makeTrajMovie2D(testSim.posTraj, testSim.sideLen, filename = "./simulation-outputs/testSimNEW.gif")

# plot kinetic and potential energies using csv file generated by dynamics library
vis.plotKEandPEandTotal("KE.csv", "PE.csv")

# # create xyz file for visualisation e.g. in vmd. default atom type is argon
# vis.writeXYZ(testSim, "./simulation-outputs/testSim2dNEW.xyz")